In [1]:
"""use this to convert .csv to coco

data is formatted as

data:
images
annos

"""

import os

root = os.getcwd()

print(root)

c:\Users\endle\Desktop\pytorch-retinanet


In [3]:
train_csv_path = r'data\train_labels.csv'
test_csv_path = r'data\test_labels.csv'

import pandas as pd

df_train = pd.read_csv(train_csv_path)
df_test = pd.read_csv(test_csv_path)

print(df_train.head())
print(df_test.head())

print('len of train: ', len(df_train))
print('len of test: ', len(df_test))



   Unnamed: 0                                   filename  width  height   
0           0          alopecia_areata_0002_Original.jpg    512     512  \
1           1          alopecia_areata_0002_Original.jpg    512     512   
2           2         alopecia_areata_0002_RandomFog.jpg    512     512   
3           3         alopecia_areata_0002_RandomFog.jpg    512     512   
4           4  alopecia_areata_0002_RandomBrightness.jpg    512     512   

             class  xmin  ymin  xmax  ymax  
0  alopecia_areata    78    71   340   262  
1  alopecia_areata   140   310   295   459  
2  alopecia_areata    78    71   340   262  
3  alopecia_areata   140   310   295   459  
4  alopecia_areata   160     8   457   331  
   Unnamed: 0                                   filename  width  height   
0           0          alopecia_areata_0662_Original.jpg    512     512  \
1           1         alopecia_areata_0662_RandomFog.jpg    512     512   
2           2  alopecia_areata_0662_RandomBrightness.j

In [4]:
# combine both dfs into one

df = pd.concat([df_train,df_test])
print(df.head())
print('len of total df: ', len(df))



   Unnamed: 0                                   filename  width  height   
0           0          alopecia_areata_0002_Original.jpg    512     512  \
1           1          alopecia_areata_0002_Original.jpg    512     512   
2           2         alopecia_areata_0002_RandomFog.jpg    512     512   
3           3         alopecia_areata_0002_RandomFog.jpg    512     512   
4           4  alopecia_areata_0002_RandomBrightness.jpg    512     512   

             class  xmin  ymin  xmax  ymax  
0  alopecia_areata    78    71   340   262  
1  alopecia_areata   140   310   295   459  
2  alopecia_areata    78    71   340   262  
3  alopecia_areata   140   310   295   459  
4  alopecia_areata   160     8   457   331  
len of total df:  11478


In [6]:
import numpy as np
import json
import pandas as pd

# use this method to convert csv to json

def convert_to_coco(data, save_json_path):
    #path = 'tensorflow2csv.csv'
    #data = pd.read_csv(path)

    images = []
    categories = []
    annotations = []

    category = {}
    category["supercategory"] = 'none'
    category["id"] = 0
    category["name"] = 'None'
    categories.append(category)

    data['fileid'] = data['filename'].astype('category').cat.codes
    data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
    data['categoryid'] = data['categoryid']+1
    data['annid'] = data.index

    def image(row):
        image = {}
        image["height"] = row.height
        image["width"] = row.width
        image["id"] = row.fileid
        image["file_name"] = row.filename
        return image

    def category(row):
        category = {}
        category["supercategory"] = 'None'
        category["id"] = row.categoryid
        category["name"] = row[2]
        return category

    def annotation(row):
        annotation = {}
        area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
        annotation["segmentation"] = []
        annotation["iscrowd"] = 0
        annotation["area"] = area
        annotation["image_id"] = row.fileid

        annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

        annotation["category_id"] = row.categoryid
        annotation["id"] = row.annid
        return annotation

    for row in data.itertuples():
        annotations.append(annotation(row))

    imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
    for row in imagedf.itertuples():
        images.append(image(row))

    catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
    for row in catdf.itertuples():
        categories.append(category(row))

    data_coco = {}
    data_coco["images"] = images
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations


    json.dump(data_coco, open(save_json_path, "w"), indent=4)

# see if it works

#save_json_path = root + "\\" 'sbl_ds_coco.json'

#convert_to_coco(df, save_json_path)
